In [1]:
import os
import numpy as np
import nltk
from keras.models import Sequential, Model
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, recurrent, Embedding,Dropout, Bidirectional, Input, merge, Flatten, Reshape
from keras.preprocessing.sequence import pad_sequences
import keras
import json
from data_utils import *
from model import *

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [58]:
SENTENCE_LENGTH = 20
VOCAB_SIZE = 10000

def load_cornell_movie_data(start_line, limit, vocab_dict):
    sentences = read_lines("movie_lines.txt", " +++$+++ ", 3, 4, start_line, limit)
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec

def load_twitter_data(start_line, limit, vocab_dict):
    sentences = read_sentences("ShortenTwitterAsciiCorpus.txt")[start_line:start_line+limit]
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec

def load_friends_data(start_line, limit, vocab_dict):
    sentences = read_lines("friends-final.txt", "	", 2, 5, start_line = 0, limit = 10000)
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec

In [9]:
#model = create_model(VOCAB_SIZE, SENTENCE_LENGTH, VOCAB_SIZE, SENTENCE_LENGTH, 100, 3)
model = create_model_with_attention(VOCAB_SIZE, SENTENCE_LENGTH, VOCAB_SIZE, SENTENCE_LENGTH, 300, 100, 3)
print(model.summary())
model.load_weights("0new_model.h5")

C:\Users\Gavin\Projects\ML\MLPractice\chatbot\model.py:36: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  output_att = merge([inputs, att_prob ], mode='mul')
C:\ProgramData\Anaconda3\lib\site-packages\keras\legacy\layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 20, 300)      3000000     input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 20, 200)      320800      embedding_2[0][0]                
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 4000)         0           bidirectional_2[0][0]            
__________________________________________________________________________________________________
repeat_vec

In [4]:
vocab_dict, idx_dict = load_vocab("vocab_dict.json")

In [ ]:
batch_size = 1000
start_point = 0000
batch = 0
round = 0
while True:
    print("Getting data:", start_point, batch_size)
    x_vec, y_vec = load_friends_data(start_point, batch_size, vocab_dict)
    
    
    if len(x_vec) == 0:
        start_point = 0
        batch = 0
        x_vec, y_vec = load_friends_data(start_point, batch_size, vocab_dict)
        round += 1 
    y = np.argmax(y_vec, axis=2)
    print(" ".join([idx_dict[x] for x in y[0] ]) )
    model.fit(x_vec, y_vec, batch_size=100, nb_epoch=100, shuffle = True)
    model.save_weights(str(batch%2)+"new_model.h5")
    start_point += batch_size
    batch += 1
    print("--------------------------------------------------------------------------")
    print("Round:", round, "Batch:", batch)
    print("--------------------------------------------------------------------------")

Getting data: 0 1000
(4885, 20, 10000)
                  there's nothing to tell he's just some guy i work with
Epoch 1/100


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


4885/4885 [==============================] - 115s 24ms/step - loss: 3.9766 - acc: 0.5478
Epoch 2/100
4885/4885 [==============================] - 115s 23ms/step - loss: 3.5227 - acc: 0.5646
Epoch 3/100
4885/4885 [==============================] - 115s 24ms/step - loss: 3.3787 - acc: 0.5679
Epoch 4/100
4885/4885 [==============================] - 115s 24ms/step - loss: 3.2779 - acc: 0.5695
Epoch 5/100
4885/4885 [==============================] - 115s 24ms/step - loss: 3.1977 - acc: 0.5709
Epoch 6/100
4885/4885 [==============================] - 115s 23ms/step - loss: 3.1317 - acc: 0.5720
Epoch 7/100
4885/4885 [==============================] - 114s 23ms/step - loss: 3.0784 - acc: 0.5728
Epoch 8/100
4885/4885 [==============================] - 125s 26ms/step - loss: 3.0149 - acc: 0.5746
Epoch 9/100
4885/4885 [==============================] - 114s 23ms/step - loss: 2.9777 - acc: 0.5749
Epoch 10/100
4885/4885 [==============================] - 115s 24ms/step - loss: 2.9152 - acc: 0.5769
E

In [54]:
#do you clean
sen = "what's your name"
sen = sen.lower().replace('?', '').replace('!', '').replace( '.', '').split(" ")
vec = sentence_to_vec([sen], vocab_dict, 'UNK', SENTENCE_LENGTH)
#print(model.predict(vec).shape)
res = model.predict(vec)
#print(res)
vec_y = np.argmax(res, axis=2)
" ".join([idx_dict[x] for x in vec_y[0] ])
#if idx_dict[x]!= 'UNK'

'                                  have i here'

In [ ]:
vocab_dict["the"]